# Fasttext

Fasttext er et en teknikk som representerer ord som vektorer, og bruker shallow neural networks til smart tekstanalyse. Det er utviklet av Facebook som et lett alternativ til deep neural networks som trenger kraftig maskinvare. 
Fasttext gir omtrent like god ytelse som tf-idf og support vector machines, gensim/word2vec, eller deep neural networks. 

Følg [instruksjoner](https://fasttext.cc/docs/en/support.html) for å komme igang. 

In [9]:
!mkdir fasttext
!mkdir data
!mkdir fasttext/data

mkdir: cannot create directory ‘fasttext’: File exists
mkdir: cannot create directory ‘data’: File exists
mkdir: cannot create directory ‘fasttext/data’: File exists


In [2]:
import os
import sys


data_dir = os.path.join('.', 'data')

print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [3]:
from lib.data import download_data

download_data(data_dir)

In [4]:
from lib.load_dataset import load_datasets

df = load_datasets(data_dir, 2015, 2015)

print(df.shape)
df.head()

Loading datasets from year 2015 to 2015
(118096, 24)


,stillingsnummer,nav_enhet_kode,registrert_dato,sistepubl_dato,statistikk_aar_mnd,offisiell_statistikk_flagg,stilling_kilde,arbeidssted_fylkesnummer,arbeidssted_fylke,arbeidssted_kommunenummer,...,yrke_grovgruppe,yrkeskode,yrke,yrkesbetegnelse,virksomhet_organisasjonsnr,virksomhet_navn,antall_stillinger,stillingstittel,statistikk_periode,stillingsbeskrivelse
0,101201412000010,101,18.12.2014,02.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Undervisning,2342,Førskolelærere,Førskolelærer,973967592,HØGSKOLEN I ØSTFOLD,1,Barnehagelærar / Førstekonsulent (14/04241),201501,# senter for framandspråk i opplæringa er det ...
1,101201412000011,101,19.12.2014,01.02.2015,201501,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Serviceyrker og annet arbeid,5413,Fengselsbetjenter,Fengselsbetjent,993315060,HALDEN FENGSEL,1,Halden fengsel - ledig fast stilling som verks...,201501,Jobbnorge ID: 108617<br><br><p dir=#LTR# align...
2,101201412000012,101,19.12.2014,16.01.2015,201501,1,Overført fra arbeidsgiver,01,Østfold,0101,...,Serviceyrker og annet arbeid,5413,Fengselsbetjenter,Fengselsbetjent,993315060,HALDEN FENGSEL,1,Halden fengsel - ledig fast stilling som verks...,201501,Jobbnorge ID: 108301<br><br><p dir=#LTR# align...
3,101201412000013,101,23.12.2014,23.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Butikk- og salgsarbeid,3322,Selgere (engros),Key Account Manager (salg),886332882,SMART INNOVATION NORWAY AS,1,Key Account Manager,201501,# har ambisjoner om å vokse og styrker teamet ...
4,101201412000014,101,23.12.2014,30.01.2015,201501,1,Annonsert i media,01,Østfold,0101,...,Ingeniør- og ikt-fag,2152,Sivilingeniører (elektronikk),Forsker (elektronikk),886332882,SMART INNOVATION NORWAY AS,1,Senior Reasearchers/Reasearchers,201501,# is now expanding its leading team with senio...


In [5]:
import re
from lib.preprocessing import remove_tags, unescape_html, remove_whitespace

non_alphanumeric_pattern = re.compile(r'\W')

# Select subset of columns
col = [
    'stillingsbeskrivelse', 
    'yrke_grovgruppe',
    'stilling_kilde',
]
df = df[col]


# Preprocess text and add some columns
df['stillingsbeskrivelse'] = df['stillingsbeskrivelse']\
    .map(remove_tags)\
    .map(unescape_html)\
    .map(remove_whitespace)\
    .map(lambda x: non_alphanumeric_pattern.sub(' ', x))\
    .map(lambda x: x.lower())
df['is_from_nav'] = df['stilling_kilde'].map(lambda x: x=='Reg av arb.giver på nav.no')
df['from_media'] = df['stilling_kilde'].map(lambda x: x=='Annonsert i media')
df['is_healthcare'] = df['yrke_grovgruppe'].map(lambda x: x=='Helse, pleie og omsorg')
df['is_industrial'] = df['yrke_grovgruppe'].map(lambda x: x=='Industriarbeid')

print(df.shape)
df.head()




(118096, 7)


,stillingsbeskrivelse,yrke_grovgruppe,stilling_kilde,is_from_nav,from_media,is_healthcare,is_industrial
0,senter for framandspråk i opplæringa er det ...,Undervisning,Annonsert i media,False,True,False,False
1,Jobbnorge ID 108617 fengsel er det ledig fas...,Serviceyrker og annet arbeid,Overført fra arbeidsgiver,False,False,False,False
2,Jobbnorge ID 108301 fengsel er det ledig fas...,Serviceyrker og annet arbeid,Overført fra arbeidsgiver,False,False,False,False
3,har ambisjoner om å vokse og styrker teamet ...,Butikk- og salgsarbeid,Annonsert i media,False,True,False,False
4,is now expanding its leading team with senio...,Ingeniør- og ikt-fag,Annonsert i media,False,True,False,False


In [8]:
import csv
from sklearn.model_selection import train_test_split

label_column = 'yrke_grovgruppe'

def make_dataframe_fasttext_format(df, label_col, size=None, random_seed=305):
    if not size:
        size = len(df)
    result = df.sample(n=size, random_state=random_seed)[[label_col, 'stillingsbeskrivelse']]
    result[label_col] = result[label_col].map(lambda x: str(x).replace(' ', '_'))
    result[label_col] = result[label_col].map(lambda x: '__label__' + str(x))
    return result


out = make_dataframe_fasttext_format(df, label_column)

def write_train_test_files_for_fasttext(df, name, test_frac=0.2):
    train_df, test_df = train_test_split(df, test_size=test_frac)
    test_df.to_csv(os.path.join('.', 'fasttext', 'data', ''.join([name, '.test'])), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE)
    train_df.to_csv(os.path.join('.', 'fasttext', 'data', ''.join([name, '.train'])), escapechar=' ', sep=' ', header=None, mode='w', index=False, quoting=csv.QUOTE_NONE)
    

write_train_test_files_for_fasttext(out, label_column)